In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from unidecode import unidecode
from sklearn.cluster import KMeans
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
#import pyLDAvis.sklearn


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


In [ ]:
df=pd.read_csv('/global/homes/m/masare/ticket-text-analysis/ticket_cleaned.csv')
df.head()

In [ ]:
text_data = df['text_data']
print(text_data)

In [ ]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
model

In [ ]:
df_val = df.text_data.apply(unidecode).to_numpy()

In [ ]:
#df_val

In [ ]:
# The sentences to encode
#sentences = [
   # "The weather is lovely today.",
  #  "It's so sunny outside!",
 #   "He drove to the stadium.",
#]

In [ ]:
# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(df_val, show_progress_bar=True)
print(embeddings.shape)
# [6944, 384]


In [ ]:
%%time
# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

In [ ]:
# Perform kmean clustering
num_clusters = 30
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(similarities)
cluster_assignment = clustering_model.labels_

In [ ]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(df_val[sentence_id])

In [ ]:
#for i, cluster in enumerate(clustered_sentences):
   # print("Cluster ", i + 1)
  #  print(cluster)
  #  print("")

In [ ]:
# Print a limited number of sentences per cluster
#for i, cluster in enumerate(clustered_sentences):
 #   print(f"Cluster {i + 1}")
 #   print(cluster[:5])  # Print only the first 5 sentences in each cluster
  #  print("")



In [ ]:

# To see the total number of sentences in each cluster
for i, cluster in enumerate(clustered_sentences):
    print(f"Cluster {i + 1} has {len(cluster)} sentences")
    print("")




In [ ]:
text_data = df_val.tolist()

In [ ]:
# Preprocessing function
def preprocess(text):
    text = text.lower()
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)


In [ ]:
# Preprocess the text data
preprocessed_data = [preprocess(text) for text in text_data]




In [ ]:
# Vectorize the text data
vectorizer = CountVectorizer(max_df=0.9, min_df=2, stop_words='english')
text_vectorized = vectorizer.fit_transform(preprocessed_data)


In [ ]:
# Apply LDA for topic modeling
num_topics = 30  # Number of topics
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_model.fit(text_vectorized)


In [ ]:
# Get the topics
def get_topics(model, vectorizer, top_n=10):
    topics = {}
    for idx, topic in enumerate(model.components_):
        topics[f'Topic {idx}'] = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-top_n - 1:-1]]
    return topics


topics = get_topics(lda_model, vectorizer)
for topic, words in topics.items():
    print(f"{topic}: {', '.join(words)}")


In [ ]:


import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from sklearn.feature_extraction.text import CountVectorizer




In [ ]:
# Prepare the pyLDAvis visualization
#import pyLDAvis.sklearn
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()
lda_vis_data = pyLDAvis.lda_model.prepare(lda_model, text_vectorized, vectorizer)


In [ ]:
# Display the visualization
pyLDAvis.display(lda_vis_data)


In [ ]:
import json


# Convert the prepared data to JSON
lda_vis_json = pyLDAvis.save_json(lda_vis_data)


# Load the JSON data
lda_vis_dict = json.loads(lda_vis_json)


# Customize the topic colors
topic_colors = ["#FF0000", "#00FF00", "#0000FF", "#FF00FF", "#00FFFF", "#FFFF00", "#FF8000", "#8000FF", "#00FF80", "#80FF00"]
for i, color in enumerate(topic_colors):
    lda_vis_dict['topic']['colors'][str(i+1)] = color


# Save the modified JSON data back
with open("lda_vis_custom.json", "w") as f:
    json.dump(lda_vis_dict, f)


# Load the modified JSON data and display the visualization
pyLDAvis.display(pyLDAvis.display_data(lda_vis_dict))




In [ ]:
#Semantic Textual Similarity
# Two lists of sentences
sentences1 = [
    "Scratch purge extension",
    "new allocation year",
    "python not working",
    "I like cheese",
]



In [ ]:
sentences2 = [
    "authentication",
    "no username",
    "mfa not working",
    
]

In [ ]:
sentences2 = [
    "my python notebook won't run",
    "new account",
    "mfa not working",
    "I hate cheese",
]

In [ ]:
# Compute embeddings for both lists
embeddings1 = model.encode(sentences1)
embeddings2 = model.encode(sentences2)


In [ ]:
# Compute cosine similarities
similarities2 = model.similarity(embeddings1, embeddings2)

In [ ]:
# Output the pairs with their score
for idx_i, sentence1 in enumerate(sentences1):
    print(sentence1)
    for idx_j, sentence2 in enumerate(sentences2):
        print(f" - {sentence2: <30}: {similarities2[idx_i][idx_j]:.4f}")
#he SentenceTransformer.similarity method returns a 3x3 matrix with the respective cosine similarity scores for all possible pairs between embeddings1 and embeddings2.

In [ ]:
####             BERTopic
#Install these packages in terminal
#pip install bertopic

# Choose an embedding backend
#pip install bertopic[flair,gensim,spacy,use]

# Topic modeling with images
#pip install bertopic[vision]

In [ ]:
titles = df['text_data']

In [ ]:
#text_data[0]

In [ ]:
#Sentence Splitter
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
sentences = [sent_tokenize(text_data) for text_data in text_data]
sentences = [sentence for doc in sentences for sentence in doc]

In [ ]:
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

In [ ]:
#Controlling Number of Topics
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)


In [ ]:
#Improving Default Representation
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")


In [ ]:
#Additional Representations
import openai
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5
prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
topic: <topic label>
"""
client = openai.OpenAI(api_key="sk-...")
openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    # "OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model,
    "POS": pos_model
}